In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from neuralforecast.core import NeuralForecast
from neuralforecast.models import NHITS, NBEATS, PatchTST
from neuralforecast.losses.numpy import mae, mse

In [ ]:
torch.cuda.is_available()

In [ ]:
Y_df = pd.read_csv('data/data_PatchTST_50.csv')
Y_df

In [ ]:
print(len(Y_df['unique_id'].unique()))

In [ ]:
Y_df['ds'] = pd.to_datetime(Y_df['ds'])
Y_df.head()

In [ ]:
Y_df['unique_id'].value_counts()

In [ ]:
n_time = len(Y_df.ds.unique())
val_size = 760
test_size = 1517

print(n_time, val_size, test_size)

Y_df.groupby('unique_id').head(2)

In [ ]:
unique_id = Y_df['unique_id'].unique()
unique_id = np.sort(unique_id)
print(len(unique_id))
unique_id

In [ ]:
# Plot previous data for one client
u_id = unique_id[0]

x_plot = pd.to_datetime(Y_df[Y_df.unique_id==u_id].ds)
y_plot = Y_df[Y_df.unique_id==u_id].y.values

x_val = x_plot[n_time - val_size - test_size]
x_test = x_plot[n_time - test_size]

fig, ax = plt.subplots(figsize=(12,8))
ax.plot(x_plot, y_plot)
ax.set_xlabel('Horodate')
ax.set_ylabel('Puissance_W')
ax.axvline(x_val, color='black', linestyle='--')
ax.axvline(x_test, color='black', linestyle='--')

plt.text(x_val, -2, 'Validation', fontsize=12)
plt.text(x_test,-2, 'Test', fontsize=12)

plt.tight_layout()

In [ ]:
horizon = 96
n_epochs = 100

# 2 state-of-the-art models and PatchTST given 14 previous days to forecast next 2 days
models = [NHITS(h=horizon,
                input_size=7*horizon,
                max_steps=n_epochs),
          NBEATS(h=horizon,
                 input_size=7*horizon,
                 max_steps=n_epochs),
          PatchTST(h=horizon,
                   input_size=7*horizon,
                   max_steps=n_epochs)]

In [ ]:
# Forecast frequency: 30 minutes
nf = NeuralForecast(models=models, freq=pd.tseries.offsets.Minute(30))

In [ ]:
# Process forecasting for the 3 models
preds_df = nf.cross_validation(df=Y_df, val_size=val_size, test_size=test_size, n_windows=None)

In [ ]:
# Display predictions
preds_df

In [ ]:
# Save trained model
nf.save('model/patchtst_model')

In [ ]:
x_true = preds_df['ds'].values
y_true = preds_df['y'].values
y_pred_nhits = preds_df['NHITS'].values
y_pred_nbeats = preds_df['NBEATS'].values
y_pred_patchtst = preds_df['PatchTST'].values

n_series = len(Y_df['unique_id'].unique())

x_true = x_true.reshape(n_series, -1, horizon)
y_true = y_true.reshape(n_series, -1, horizon)
y_pred_nhits = y_pred_nhits.reshape(n_series, -1, horizon)
y_pred_nbeats = y_pred_nbeats.reshape(n_series, -1, horizon)
y_pred_patchtst = y_pred_patchtst.reshape(n_series, -1, horizon)

print(y_true.shape)
print(y_pred_nhits.shape)

In [ ]:
# Plot forecastings for client 'i'
fig, ax = plt.subplots(figsize=(16,8))
i = 0

ax.plot(x_true[i, 0, :], y_true[i, 0, :], label='True')
ax.plot(x_true[i, 0, :], y_pred_nhits[i, 0, :], label='N-HiTS', ls='--')
ax.plot(x_true[i, 0, :], y_pred_nbeats[i, 0, :], label='N-BEATS', ls=':')
ax.plot(x_true[i, 0, :], y_pred_patchtst[i, 0, :], label='PatchTST', ls='-.')
ax.set_ylabel('Puissance_W')
ax.set_xlabel('Horodate')
ax.legend(loc='best')

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True, figsize=(16, 8))
i1 = 0
i2 = 1

ax[0].plot(x_true[i1, 0, :], y_true[i1, 0, :], label='True')
ax[0].plot(x_true[i1, 0, :], y_pred_patchtst[i1, 0, :], label='PatchTST', ls='-.')
ax[0].set_ylabel('Puissance [W]')
ax[0].legend(loc='best')
ax[0].set_title(f'client {unique_id[i1]}')
ax[1].plot(x_true[i2, 0, :], y_true[i2, 0, :], label='True')
ax[1].plot(x_true[i2, 0, :], y_pred_patchtst[i2, 0, :], label='PatchTST', ls='-.')
ax[1].set_ylabel('Puissance [W]')
ax[1].set_xlabel('Horodate')
ax[1].legend(loc='best')
ax[1].set_title(f'client {unique_id[i2]}')

plt.tight_layout()

In [ ]:
# Compute MSE score on entire prediction for the 3 models
data = {'N-HiTS': [mae(y_pred_nhits, y_true), mse(y_pred_nhits, y_true)],
       'N-BEATS': [mae(y_pred_nbeats, y_true), mse(y_pred_nbeats, y_true)],
       'PatchTST': [mae(y_pred_patchtst, y_true), mse(y_pred_patchtst, y_true)]}

metrics_df = pd.DataFrame(data=data)
metrics_df.index = ['mae', 'mse']

metrics_df.style.highlight_min(color='lightgreen', axis=1)